In [1]:
import os
import numpy as np
from tqdm import tqdm #progress bar
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#make a list of wells in a 384-well plate
plate_columns=list(range(25))[1:]
plate_rows=["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P"]
wells=[]
for r in plate_rows:
    for c in plate_columns:
        wells.append(r+str(c))
        
wells96 = []
for r in plate_rows[0:8]:
    for c in plate_columns[0:12]:
        wells96.append(r+str(c))


In [13]:
"""Note: units for ALL volumes is nL"""

#lets get sample information about the experiment you are running


doses = 5 #number of points in dose curve
doses2 = 6 #number points in "second" antigen dose curve
highest_dose = 675 #uM
vol_cellextract = 2000 #volume of cell extract in nL
vol_antigen = 2000 #volume of antigen in nL
all_variants = wells96[0:29] + ["purified nCov19-NS 15uM", "purified nCov19-NS 7.5uM", "purified nCov19-NS 3.75uM", "purified nCov19-NS 1.875uM"] #list of variants for source plate
#variants = wells96[0:6] + wells96[7:17] + ["purified nCov19-NS 15uM", "purified nCov19-NS 7.5uM", "purified nCov19-NS 3.75uM", "purified nCov19-NS 1.875uM"] #list of variants we want to test
variants = wells96[0:6] + wells96[7:9] + wells96[17:29]
num_samples = len(variants)
dna_vol = 60000 #really sensor volume here

print(variants)
print(all_variants)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A8', 'A9', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5']
['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'purified nCov19-NS 15uM', 'purified nCov19-NS 7.5uM', 'purified nCov19-NS 3.75uM', 'purified nCov19-NS 1.875uM']


In [4]:
#lets make a dataframe that functions as a plate map

def make_plate():
    plate_columns=list(range(25))[1:]
    plate_rows=["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P"]
    plate_df = pd.DataFrame(index=plate_rows, columns=plate_columns)
    return(plate_df)

In [5]:
"""siddharth says to keep everything in terms of lists of 384 length, make lists of NaN's and add in values, then in
the back end it 
"""

source_sample_names = ['NaN']*384
source_sample_vol = [0]*384
dest_extract_vol = [0]*384
dest_dna_vol = [0]*384
dest_antigen_vol = [0]*384
dest_antigen2_vol = [0]*384
dest_pbs_vol = [0]*384
dest_sample_names = ['NaN']*384


#lets start with adding the DNA samples to the source plate
i=0
for v in all_variants:
    source_sample_names[i]=v
    source_sample_vol[i]=dna_vol
    i+=2

#i=192 #row I
i=216 #row J
#i = 264 #row L

''' 
#now let's calculate how much MM we will need, then add it to the source plate
master_mix = num_samples*doses*vol_cellextract
wells_mm_needed = master_mix//50000 + 1


while wells_mm_needed>1:
    source_sample_names[i]='cell extract master mix'
    source_sample_vol[i]=55000
    i+=1
    wells_mm_needed-=1

source_sample_names[i]='cell extract master mix'
source_sample_vol[i]=15000+(master_mix%40000)
i+=1

'''  

#print(source_sample_names)
#print(source_sample_vol)

" \n#now let's calculate how much MM we will need, then add it to the source plate\nmaster_mix = num_samples*doses*vol_cellextract\nwells_mm_needed = master_mix//50000 + 1\n\n\nwhile wells_mm_needed>1:\n    source_sample_names[i]='cell extract master mix'\n    source_sample_vol[i]=55000\n    i+=1\n    wells_mm_needed-=1\n\nsource_sample_names[i]='cell extract master mix'\nsource_sample_vol[i]=15000+(master_mix%40000)\ni+=1\n\n"

In [6]:
#now let's add the antigen & PBS wells


amnt_antigen_to_add = [2000, 675, 225, 75, 25, 0] #approx 1:3 dilutions, in nL
#amnt_antigen_to_add = [2000, 0]
amnt_pbs_to_add = []
for a in amnt_antigen_to_add:
    amnt_pbs_to_add.append(vol_antigen-a)
    
#adding antigen to source plate
total_antigen_vol = sum(amnt_antigen_to_add)*num_samples
wells_antigen_needed = total_antigen_vol//40000 + 1

while wells_antigen_needed>1:
    source_sample_names[i]='antigen'
    source_sample_vol[i]=55000
    i+=1
    wells_antigen_needed-=1

source_sample_names[i]='antigen'
source_sample_vol[i]=15000+(total_antigen_vol%40000)
i+=1

#adding 2nd antigen to source plate
total_antigen2_vol = sum(amnt_antigen_to_add[0:doses2])*num_samples
wells_antigen2_needed = total_antigen2_vol//40000 + 1

while wells_antigen2_needed>1:
    source_sample_names[i]='antigen2'
    source_sample_vol[i]=55000
    i+=1
    wells_antigen2_needed-=1

source_sample_names[i]='antigen2'
source_sample_vol[i]=15000+(total_antigen2_vol%40000)
i+=1

#adding pbs to source plate
total_pbs_vol = sum(amnt_pbs_to_add)*num_samples + amnt_pbs_to_add[0]*num_samples
wells_pbs_needed = total_pbs_vol//40000 + 1

while wells_pbs_needed>1:
    source_sample_names[i]='PBS'
    source_sample_vol[i]=55000
    i+=1
    wells_pbs_needed-=1

source_sample_names[i]='PBS'
source_sample_vol[i]=15000+(total_pbs_vol%40000)
i+=1

print(source_sample_names)
print(source_sample_vol)


['A1', 'NaN', 'A2', 'NaN', 'A3', 'NaN', 'A4', 'NaN', 'A5', 'NaN', 'A6', 'NaN', 'A7', 'NaN', 'A8', 'NaN', 'A9', 'NaN', 'A10', 'NaN', 'A11', 'NaN', 'A12', 'NaN', 'B1', 'NaN', 'B2', 'NaN', 'B3', 'NaN', 'B4', 'NaN', 'B5', 'NaN', 'B6', 'NaN', 'B7', 'NaN', 'B8', 'NaN', 'B9', 'NaN', 'B10', 'NaN', 'B11', 'NaN', 'B12', 'NaN', 'C1', 'NaN', 'C2', 'NaN', 'C3', 'NaN', 'C4', 'NaN', 'C5', 'NaN', 'purified nCov19-NS 15uM', 'NaN', 'purified nCov19-NS 7.5uM', 'NaN', 'purified nCov19-NS 3.75uM', 'NaN', 'purified nCov19-NS 1.875uM', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN'

In [7]:
#dest_extract_vol = [0]*384
#dest_dna_vol = [0]*384
#dest_antigen_vol = [0]*384
#dest_pbs_vol = [0]*384

place = 0



for v in variants:
    
    
    for d in range(doses):
        
        dest_dna_vol[place] = 2000 ##calculate this volume
        dest_antigen_vol[place] = amnt_antigen_to_add[d]
        dest_pbs_vol[place] = amnt_pbs_to_add[d]
        dest_sample_names[place] = v + "_dose" + str(d+1)
        place+=1
        
    for d in range(doses2):
        
        dest_dna_vol[place] = 2000 ##calculate this volume
        dest_antigen2_vol[place] = amnt_antigen_to_add[d]
        dest_pbs_vol[place] = amnt_pbs_to_add[d]
        dest_sample_names[place] = v + "_dose" + str(d+doses+1)
        place+=1
        

                
        

        
print(dest_sample_names)
print(dest_antigen_vol)
print(dest_antigen2_vol)

['A1_dose1', 'A1_dose2', 'A1_dose3', 'A1_dose4', 'A1_dose5', 'A1_dose6', 'A1_dose7', 'A1_dose8', 'A1_dose9', 'A1_dose10', 'A1_dose11', 'A2_dose1', 'A2_dose2', 'A2_dose3', 'A2_dose4', 'A2_dose5', 'A2_dose6', 'A2_dose7', 'A2_dose8', 'A2_dose9', 'A2_dose10', 'A2_dose11', 'A3_dose1', 'A3_dose2', 'A3_dose3', 'A3_dose4', 'A3_dose5', 'A3_dose6', 'A3_dose7', 'A3_dose8', 'A3_dose9', 'A3_dose10', 'A3_dose11', 'A4_dose1', 'A4_dose2', 'A4_dose3', 'A4_dose4', 'A4_dose5', 'A4_dose6', 'A4_dose7', 'A4_dose8', 'A4_dose9', 'A4_dose10', 'A4_dose11', 'A5_dose1', 'A5_dose2', 'A5_dose3', 'A5_dose4', 'A5_dose5', 'A5_dose6', 'A5_dose7', 'A5_dose8', 'A5_dose9', 'A5_dose10', 'A5_dose11', 'A6_dose1', 'A6_dose2', 'A6_dose3', 'A6_dose4', 'A6_dose5', 'A6_dose6', 'A6_dose7', 'A6_dose8', 'A6_dose9', 'A6_dose10', 'A6_dose11', 'A8_dose1', 'A8_dose2', 'A8_dose3', 'A8_dose4', 'A8_dose5', 'A8_dose6', 'A8_dose7', 'A8_dose8', 'A8_dose9', 'A8_dose10', 'A8_dose11', 'A9_dose1', 'A9_dose2', 'A9_dose3', 'A9_dose4', 'A9_dose5', '

In [8]:
source_plate_df = make_plate()

for well in range(384):
    if source_sample_vol[well] > 0:
        row = well//24
        column = well%24
        source_plate_df.iat[row,column] = source_sample_names[well] + ": " + str(source_sample_vol[well]) + "nL"
    
source_plate_df.to_csv('20230731_ncov19_curves_source_plate.csv')   
#print(source_plate_df)

In [9]:
half_plate = 192


In [10]:
echo_protocol = pd.DataFrame(columns=["Sample Name", "Source Plate Name", "Source Well", "Destination Well", 
                                      "Transfer Volume", "Destination Plate Name", "Source Plate Type"])

def add_line(sample, source, dest, vol, df):
    new = pd.Series([sample,'af_src001',source,dest,vol,"dest_plate","384PP_AQ_BP"], index=df.columns)
    return(df.append(new, ignore_index=True))


'''
for well in range(384):
    #lets first add cell extract to the protocol
    if dest_extract_vol[well] > 0:
        #first need to find the well in the source plate
        for x in range(192,384):
            if source_sample_names[x]=='cell extract master mix' and source_sample_vol[x]>15000+dest_extract_vol[well]:
                source_well = wells[x]
                source_sample_vol[x] = source_sample_vol[x]-dest_extract_vol[well]
                break          
        echo_protocol = add_line("cell_extract", source_well, wells[well], dest_extract_vol[well], echo_protocol)
'''

    #now lets add the dna to the protocol
for well in range(384):
    if dest_dna_vol[well] > 0:
        sample_dna = dest_sample_names[well].split('_')[0]    
        for x in range(192):
            if source_sample_names[x]==sample_dna:
                source_well = wells[x]
                break
        echo_protocol = add_line(sample_dna, source_well, wells[well], dest_dna_vol[well], echo_protocol)        
        
#next let's add the pbs to the protocol
for well in range(384):
    if dest_pbs_vol[well] > 0:
        #first need to find the well in the source plate
        for x in range(192,384):
            if source_sample_names[x]=='PBS' and source_sample_vol[x]>15000+dest_pbs_vol[well]:
                source_well = wells[x]
                source_sample_vol[x] = source_sample_vol[x]-dest_pbs_vol[well]
                break          
        echo_protocol = add_line("PBS", source_well, wells[well], dest_pbs_vol[well], echo_protocol)

        
#lastly add the antigen
for well in range(384):
    if dest_antigen_vol[well] > 0:
        #first need to find the well in the source plate
        for x in range(192,384):
            if source_sample_names[x]=='antigen' and source_sample_vol[x]>15000+dest_antigen_vol[well]:
                source_well = wells[x]
                source_sample_vol[x] = source_sample_vol[x]-dest_antigen_vol[well]
                break          
        echo_protocol = add_line("antigen", source_well, wells[well], dest_antigen_vol[well], echo_protocol)

#lastly add the 2nd antigen
for well in range(384):
    if dest_antigen2_vol[well] > 0:
        #first need to find the well in the source plate
        for x in range(192,384):
            if source_sample_names[x]=='antigen2' and source_sample_vol[x]>15000+dest_antigen2_vol[well]:
                source_well = wells[x]
                source_sample_vol[x] = source_sample_vol[x]-dest_antigen2_vol[well]
                break          
        echo_protocol = add_line("antigen2", source_well, wells[well], dest_antigen2_vol[well], echo_protocol)

print(echo_protocol)

#export protocol to csv
echo_protocol.to_csv('20230731_ncov19_curves_echo_protocol.csv')   



    Sample Name Source Plate Name Source Well Destination Well  \
0            A1         af_src001          A1               A1   
1            A1         af_src001          A1               A2   
2            A1         af_src001          A1               A3   
3            A1         af_src001          A1               A4   
4            A1         af_src001          A1               A5   
..          ...               ...         ...              ...   
595    antigen2         af_src001          J4              I23   
596    antigen2         af_src001          J4              I24   
597    antigen2         af_src001          J4               J1   
598    antigen2         af_src001          J4               J2   
599    antigen2         af_src001          J4               J3   

    Transfer Volume Destination Plate Name Source Plate Type  
0              2000             dest_plate       384PP_AQ_BP  
1              2000             dest_plate       384PP_AQ_BP  
2              200

In [11]:
for x in range(192,384):
    print(wells[x])

I1
I2
I3
I4
I5
I6
I7
I8
I9
I10
I11
I12
I13
I14
I15
I16
I17
I18
I19
I20
I21
I22
I23
I24
J1
J2
J3
J4
J5
J6
J7
J8
J9
J10
J11
J12
J13
J14
J15
J16
J17
J18
J19
J20
J21
J22
J23
J24
K1
K2
K3
K4
K5
K6
K7
K8
K9
K10
K11
K12
K13
K14
K15
K16
K17
K18
K19
K20
K21
K22
K23
K24
L1
L2
L3
L4
L5
L6
L7
L8
L9
L10
L11
L12
L13
L14
L15
L16
L17
L18
L19
L20
L21
L22
L23
L24
M1
M2
M3
M4
M5
M6
M7
M8
M9
M10
M11
M12
M13
M14
M15
M16
M17
M18
M19
M20
M21
M22
M23
M24
N1
N2
N3
N4
N5
N6
N7
N8
N9
N10
N11
N12
N13
N14
N15
N16
N17
N18
N19
N20
N21
N22
N23
N24
O1
O2
O3
O4
O5
O6
O7
O8
O9
O10
O11
O12
O13
O14
O15
O16
O17
O18
O19
O20
O21
O22
O23
O24
P1
P2
P3
P4
P5
P6
P7
P8
P9
P10
P11
P12
P13
P14
P15
P16
P17
P18
P19
P20
P21
P22
P23
P24


In [12]:
#lastly add the antigen